## A frequency ranked list of economics vocabulary

### Aim of project:
To help a friend improve her economics specific English vocabulary in an efficient way. 

### End result of project:
1. A <a href="https://github.com/pvonglehn/economics_vocab/blob/master/economics_vocab.txt">list</a> of the most common, economics specific English words which don't have Spanish cognates. That is, words which are common in economics texts but uncommon in general texts and which cannot be easily guessed by a Spanish speaker. 
<br><br>
2. Anki flashcards deck for studying these words with example sentences (still to do)
 


### Problem:
A friend is preparing for an English exam as part of her studies to become an economist for the civil service in Spain. Part of the exam will be based on an article from the International Monetary Fund <a href="https://www.imf.org/external/pubs/ft/fandd/">finance and development magazine</a>, or a similar publication. She finds that she is lacking much of the economics specific vocabulary necessary to understand these articles. 

The classic strategy for improving vocabulary is to read a lot and look up words that you don't know. However, this approach is very inefficient. 

Let's consider an example.
The student reads the sentence:<br>
'Higher <strong>wages</strong> in China make <strong>offshoring</strong> less attractive.'

The student may not know the meaning of 'wages' or 'offshoring' so their instinct might be to look up and learn both. What the student doesn't know however, is that while the word 'wage' is very common in financial texts and definitely worth learning,  the word 'offshoring' is much less common, so it is not worth the student's effort learning, at least as long as there are many more useful words they could learn first.

### Project results:

Below is a table showing the first 5 words in the final list. The imf_rank indicates each word's rank based on its frequency of occurrence in the imf magazine. This corpus was compiled during this project and consists of over two million tokens (words) from over 100,000 sentences from over 1000 articles. The general rank indicates the word's position on a list of 5000 English words ordered by their frequency of occurence in a large corpus constructed from a wide range of English texts. This list was downloaded from https://www.wordfrequency.info/ 

This list only contains words which are more common in the economics corpus than in general English, so words like 'the','and' etc. don't appear. We can see that the top words in our list occur much more frequently in the economics corpus than in general English. This indicates that these words are very important to know in order to understand these texts, but students may not know them because they are relatively uncommon in general texts that they will mostly have been exposed to in their English studies. 

<br><em>A general_rank of 1000000 means that the word is not on the list from the general corpus.</em>


In [1196]:
economics_vocab_no_cognates.head(10)

,imf_rank,imf_freq,general_rank
currency,136,2328,3297
poverty,146,2123,2080
spending,175,1853,2082
inequality,206,1556,1000000
banking,228,1432,3476
macroeconomic,241,1356,1000000
deficit,289,1134,2142
wage,305,1070,2300
unemployment,325,1014,3089
commodity,329,1004,4038


### Removing words with Spanish cognates

Below is the table including words with Spanish cognates. The meanings of 'percent', 'fiscal' and 'export' etc could all be easily guessed by a Spanish speaker. Of the top 1000 words on this list, 40% of the words have Spanish cognates, so it is really worth excluding them from our list to save the student's time and effort. 

In [1195]:
w_cognate.head(10)

,imf_rank,imf_freq,general_rank,spanish_cognate
percent,32,6845,1000000,por ciento
fiscal,98,2952,3212,fiscal
currency,136,2328,3297,-
poverty,146,2123,2080,-
export,157,2036,3062,exportar
monetary,170,1892,1000000,monetario
spending,175,1853,2082,-
inflation,181,1823,2830,inflación
inequality,206,1556,1000000,-
finance,207,1551,2492,financiar







## Procedural overview
1. Generate a text corpus by web scraping economics articles 
2. Split the text into sentences and then tag each word by its part of speech (verb, noun etc.)
3. Convert the words into their lemmas (dictionary entry forms). e.g. running -> run
4. Create a frequency table of the words and put them in rank order
5. Get an existing ranked list of words from a corpus of general English
6. Produce a list of words which have a higher rank in the economics corpus than in the general corpus
7. Remove words which have Spanish cognates e.g. the economy - la economía
8. Get example sentences with Spanish translation for each word
9. Turn these example sentences into flash cards for studying

## Tools used
Beautiful Soup for web scraping<br>
NLTK for natural language processing<br>
pandas for data manipulation<br> 



## To do/ improvements to be made/ features to add

Get example sentences and make Anki flashcards.<br>
Get more articles e.g. from 'the economist'.
<br>Use an existing web crawling library such as https://scrapy.org/ to crawl entire sites in the future.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [552]:
# Web crawling function to get all links from a webpage using beautiful soup

# WARNING: Take care before reusing this function. It is one of my first attempts at 
# writing a function to scrape multiple pages and is quite hacky. It works for this case but may not
# work as desired in other cases.

# Arguments are:
# url: starting url
# base_url: the url of website homepage
# must_include: regex that the links must include (default is the wildcard character ".")
# must_not_include: regex that the links must not include (default is the nonsense string "xasdfcasdf" )

def get_all_links(url,base_url,existing_links,must_include = ".",must_not_include = "xasdfcasdf"):
    content = requests.get(url).content
    soup = BeautifulSoup(content,'lxml')
    links = []
    existing_links = existing_links.copy() # this copy is required to prevent modifying the original list (side effect)
    for anchor in soup.findAll("a"):
        # if the anchor element isn't doesn't have a href, it isn't a proper url, so skip to next anchor
        try:
            link = anchor['href']
        except:
            continue 
            
        full_link = None 
        if re.search("^http",link): # if link starts with http it is either an external link or internal with full url
            if base_url in link: # exclude links to external sites
                full_link = link 
        elif re.search("^/",link): # if link is an internat link to the base_url, create full link from base_url
            full_link = (base_url + "/" + anchor['href'])
        else:
            full_link = (url + "/" + anchor['href']) # if link is
            
        # filter out links based on various conditions    
        if ((full_link not in existing_links) # ignore links that have already been found
             and full_link is not None
             and (not re.search("#",full_link)) # filter out links to id's on the same page
             and (not re.search("htm.*htm",full_link)) # this is hack because of some buggy behaviour - should fix
             and  re.search("htm$",full_link) 
             and re.search(must_include,full_link)
             and (not re.search(must_not_include,full_link))):
            
            # add to list of links found on the whole site so far
            # (the existing_links variable here has local scope only)
            existing_links.append(full_link)
            
            # add to list of links found within this function invokation
            links.append(full_link)
            
    return links


In [576]:
%%time
# Get all internal links from the imf finance and development publications website

f = open("imf_links.txt","w") # file to save the links

base_url = "https://www.imf.org"
must_not_include = "fandd/spa|fandd/fre|fandd/rus|fandd/chi|fandd/ara|fandd/ger"
all_links = []

# The online magazine is published quaterly
# loop over the years and quaters
for year in range(1996,2019):
    for month in ("03","06","09","12"):
        must_include = "external/pubs/ft/fandd/{}/{}".format(year,month) # only get links from current edition
        current_existing = [] # initialise list of links that have been visited within this loop
        current_existing.append("https://www.imf.org/external/pubs/ft/fandd/{}/{}".format(year,month))
        for link in current_existing:
            # get all new links from within this link and add them to the list of links already found
            current_existing.extend(get_all_links(link,base_url,current_existing,must_include,must_not_include))
        all_links.extend(current_existing)
        for link in current_existing:
            f.write((link + "\n"))

f.close()


CPU times: user 1min 3s, sys: 3.35 s, total: 1min 6s
Wall time: 6min 51s


In [577]:
%%time

# Visit each link
# Extract all text
# Break the text into sentences with nltk (natural language tool kit) sentence tokenizer 

import nltk
all_sentences = []
for link in all_links:
    content = requests.get(link).content
    soup = BeautifulSoup(content,'lxml')
    text = soup.body.text
    sentences = nltk.sent_tokenize(text) # this splits the text into sentences
    all_sentences.extend(sentences)

CPU times: user 1min 9s, sys: 3.1 s, total: 1min 12s
Wall time: 4min 4s


In [578]:
# Exclude sentences which are shorter than 6 words
# As they are probably not proper sentences

long_sentences = []
for sentence in all_sentences:
    if sentence.count("\n") < 2:
        if len(sentence.split()) > 6:
            long_sentences.append(sentence)

f = open("imf_sentences.txt","w")
for line in long_sentences:
    f.write((line + "\n"))
f.close()

In [822]:
sentence_count = len(long_sentences)
word_count = pd.Series(long_sentences).apply(lambda x : len(x.split())).sum()
words_per_sentence = pd.Series(long_sentences).apply(lambda x : len(x.split())).mean()
print("sentence count = {}\ntotal word count = {}\nmean words per sentence = {:.2f}"
      .format(sentence_count,word_count,words_per_sentence))
      

sentence count = 101541
total word count = 2470329
mean words per sentence = 24.33


In [87]:
# There are libraries and functions that we will need to turn the
# words into their lemmas (dictionary forms) https://en.wikipedia.org/wiki/Lemma_(morphology)
# E.g. running will be turned into run and played into play
# To do this we need to:

# 1. tokenize the sentences (break them up into words)
# 2. tag the parts of speech for each token (word) e.g. verb, adjective
# 3. lemmatize the tokens (turn into dictionary form)

# part of speech to the lemmatizer
# Full sentences need to be passed to the parts of speech tagger in order to tag them accurately
# If the word "play" is given in isolation, it is ambiguous if it is a verb or a noun,
# but if you give the pos tagger a full sentence, it can determine from context
# e.g. I am going to play (verb) tennis. I am going to see a play (noun).

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

# This function converts the parts of speech tags from nltk pos tagger 
# To POS tags that are compatible with the wordnet lemmatizer
# function adapted from: 
# https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    else:
        return None
    

[nltk_data] Downloading package wordnet to /Users/pv7409/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/pv7409/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [629]:
%%time

all_tokens = []
all_lemmas = []
for sentence in long_sentences:
    tokens = nltk.word_tokenize(sentence)
    tokenized = nltk.pos_tag(tokens)
    for i, token in enumerate(tokenized):
        # Filter out proper nouns (words with capital letters that aren't at the beginning of a sentence)
        if token[0].islower() or i == 0:
            word = token[0].lower()
            wordnet_pos = get_wordnet_pos(token[1])
        #print(word,wordnet_pos)
            if wordnet_pos is None:
                all_lemmas.append(word)
                all_tokens.append(word)
            else:
                all_lemmas.append(lemmatizer.lemmatize(word,wordnet_pos))
                all_tokens.append(word)

# write out lemmes to text file
f = open("all_lemmas.txt","w")
for i in all_lemmas:
    f.write((i+"\n"))
f.close()

CPU times: user 3min 25s, sys: 4.73 s, total: 3min 30s
Wall time: 3min 39s


In [635]:
# No we'll process the lemmas a little

all_lemma_series = pd.Series(all_lemmas)

#remove punctuation
all_lemma_series = all_lemma_series[~all_lemma_series.str.contains("\W")]

# remove proper names
propernames = pd.Series(open("/usr/share/dict/propernames","r").read().split("\n"))
propernames = propernames.apply(lambda x: x.lower())
propernames = set(propernames)
#all_lemma_series = all_lemma_series[~all_lemma_series.isin(propernames)]

#get rid of numbers
all_lemma_series = all_lemma_series[~all_lemma_series.str.contains("\d")]


In [826]:
# Create a frequency table for each lemma
# Give each lemma a rank

lemma_frequencey = all_lemma_series.value_counts()
ranked = pd.Series(lemma_frequencey.index)
ranks = pd.DataFrame(list(range(1,len(lemma_frequencey)+1)))
ranks.index = lemma_frequencey.index
ranks['freq'] = lemma_frequencey

In [827]:
ranks.head()

,0,freq
the,1,149311
of,2,85426
and,3,82856
be,4,74219
to,5,69129


Here is a list of the 5000 most frequent words in English from https://www.wordfrequency.info/
<br>Note that although there are 5000 entries in the list, there are only 4353 unique words,
as sometimes the same word has several entries because it appears as a different part of speech 
<br>e.g. 'light' appears as a noun (the light at the end of the tunnel) and as an adjective (a light breakfast)

In [828]:
# Get list of English words by frequency, generated from 14 billion word intente corpus
# https://www.wordfrequency.info/ 
f = open("5000_eng_words.txt","r")
freq_5000 = f.read().split("\n")
freq_5000 = freq_5000[1:] # ignore header
freq_5000_list = pd.Series(freq_5000).str.lower()

#remove duplicated words in freq_5000_list
freq_5000_list = pd.Series(freq_5000_list).unique()

# give each word in the general corpus frequency list a rank
freq5000df = pd.DataFrame(list(range(1,len(freq_5000_list)+1)))
freq5000df.index = freq_5000_list

In [829]:
freq5000df.head()

,0
the,1
be,2
and,3
of,4
a,5


In [830]:
# merge the general corpus frequency data frame with our economics vocab frequency list

ranks = ranks.merge(freq5000df,how="left",left_index=True,right_index=True)
ranks.columns = "imf_rank","imf_freq","general_rank"
ranks = ranks.sort_values("imf_rank")

# if word not in general corpus list, give it rank of 1000000
ranks.loc[ranks["general_rank"].isnull(),["general_rank"]] = 1000000 
ranks["general_rank"] = ranks["general_rank"].astype(int) # turn back into integers


In [831]:
ranks.head()

,imf_rank,imf_freq,general_rank
the,1,149311,1
of,2,85426,4
and,3,82856,3
be,4,74219,2
to,5,69129,7


In [832]:
# exclude short words
large_words = pd.Series(ranks.index)
large_words = large_words[large_words.apply(lambda x : len(x) > 3)]
ranks = ranks.loc[list(large_words)]

In [851]:
my_series = pd.Series(long_sentences)
my_series[my_series.str.contains("estuary")].values

array(['Work evaporated at the port on the south shore of the giant estuary, the Rio de la Plata.'],
      dtype=object)

In [1086]:
# list of spanish/english cognates downloaded from: http://cognates.org/pdf/mfcogn.pdf
# massage the text file (formatting was messed up when converted from pdf)
f = open("cognates.txt","r").read()
f = re.sub(r'\((.*?)\)',",",f)
f = re.sub("por ciento","porciento",f)
f = re.sub("se relajó","serelajó",f)
f = re.sub("en el presente","enelpresente",f)
f = re.sub("ex prefix","prefix",f)
f = re.sub("ex prefijo","prefix",f)
f = re.sub("soul, música","soulmúsica",f)
f = re.sub("substituir v. 5/sustituir","substituir/sustituir",f)
f = re.sub("rock n' roll","rock'n'roll",f)
f = re.sub("El Salvador","ElSalvador",f)
f = re.sub("valuación, avalúo","valuación/avalúo",f)
f = re.sub("prefix","",f)
f = re.sub("intj.","",f)

f = re.sub(r'\d',",",f)
f = re.split("PMF|MFW| |conj\.|v\.|adj\.|n\.|s\.|adv\.|,|abbr\.|abr\.|prep\.",f)
to_remove = "Cognate","org","","clic","prefijo","prefix"
for item in to_remove:
    while item in f: f.remove(item)
f[f.index("porciento")] = "por ciento"
f[f.index("serelajó")] = "se relajó"
f[f.index("enelpresente")] = "en el presente"
f[f.index("soulmúsica")] = "soul música"




In [1091]:
cognate_list = []
for i in range(0,len(f)-1,2):
    cognate_list.append([f[i],f[i+1]])
cognate_df = pd.DataFrame(cognate_list)
cognate_df = pd.DataFrame(cognate_df)
cognate_df.columns = "english","spanish"
cognate_df.to_csv("spanish_english_cognates.csv",index=False)

In [1096]:
cognate_df.sample(5)

,english,spanish
3373,servant,sirviente
687,completed,completado
315,attribute,atributo
658,communist,comunista
308,attitude,actitud


In [1113]:
#economics_vocab = ranks[(ranks["imf_rank"] > ranks["general_rank"] + 500) & (ranks["general_rank"] < 10000)]
economics_vocab = ranks[((ranks["imf_rank"] ) < ranks["general_rank"]) & (ranks["general_rank"] > 2000)]

In [1118]:
s = pd.Series(economics_vocab.index)
no_cognates = list(s[~s.isin(cognate_df["english"])])
have_cognates = list(s[s.isin(cognate_df["english"])])
economics_vocab_no_cognates = economics_vocab.loc[no_cognates]
economics_vocab_with_cognates = economics_vocab.loc[have_cognates]

In [1184]:
cog_df = pd.DataFrame(cognate_df["spanish"])
cog_df.index = cognate_df["english"]

In [1190]:
w_cognate = economics_vocab.merge(cog_df,how="left",left_index=True,right_index=True)
w_cognate = w_cognate.sort_values("imf_rank")
w_cognate.loc[w_cognate['spanish'].isnull(),['spanish']] = "-"

In [1193]:
w_cognate = w_cognate.rename({"spanish":"spanish_cognate"},axis=1)

In [1129]:
pd.options.display.max_rows=100
economics_vocab_with_cognates


,imf_rank,imf_freq,general_rank
percent,32,6845,1000000
fiscal,98,2952,3212
export,157,2036,3062
monetary,170,1892,1000000
inflation,181,1823,2830
finance,207,1551,2492
advanced,220,1473,2483
external,240,1362,2536
shock,264,1240,2050
economist,285,1150,2650


In [1130]:
pd.options.display.max_rows=10
economics_vocab_no_cognates[:1000]


,imf_rank,imf_freq,general_rank
currency,136,2328,3297
poverty,146,2123,2080
spending,175,1853,2082
inequality,206,1556,1000000
banking,228,1432,3476
...,...,...,...
paul,3146,41,1000000
hurdle,3152,41,1000000
isolation,3154,41,3753
societal,3155,41,1000000


In [1133]:
f = open("economics_vocab.txt","w")
f.write("""#List of 2000 economics related words 
#Ranked by frequency of occurrence in the imf finance and development magazine
#https://www.imf.org/external/pubs/ft/fandd/
#Words with Spanish cognates removed\n""")
for i in economics_vocab_no_cognates[:2000].index:
    f.write((i+"\n"))
f.close()

In [1198]:
df.head()

,1,2,3,4,5
0,,,,,
1,cmn,我們試試看！,sysko,\N,2010-03-14 19:46:23
2,cmn,我该去睡觉了。,fucongcong,\N,2010-01-01 15:23:53
3,cmn,你在干什麼啊？,sysko,\N,2010-06-30 11:54:11
4,cmn,這是什麼啊？,Martha,2008-09-08 09:20:59,2011-02-27 12:01:04
5,cmn,今天是６月１８号，也是Muiriel的生日！,Zifre,\N,2011-08-17 19:03:02


In [1197]:
# Load in the Tatoeba database  into pandas DataFrame
# Note: file is actually tab seperated, although named .csv
# I haven't uploaded this file to github because it is very large, you will need to download it from:
# https://tatoeba.org/eng/downloads
df = pd.read_csv("/Users/pv7409/Spanish/verbs_w_prepositions/sentences_detailed.csv",sep="\t",index_col=0,header=None)


# This produced a FutureWarning. I'm not sure why this warning occured. Everything has loaded correctly now 
# but the warning means that if numpy or python is upgraded there could be issues 
# (This is unlikely to cause problems in my opinion)

/Users/pv7409/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [1223]:
phrases = []
for word in economics_vocab_no_cognates.index[:500]:
    phrases.append(df.loc[df[2].str.contains(word)].head(1))

KeyboardInterrupt: 

In [1232]:
collect = phrases[5]
for i in phrases:
    collect = collect.append(i)

In [1236]:
collect

,1,2,3,4,5
0,,,,,
21981,eng,The exchange rates for foreign currency change daily.,Swift,\N,2010-12-03 19:03:06
1533,eng,Rye was called the grain of poverty.,Zifre,\N,2011-01-23 01:21:24
1959,eng,"My roommate is prodigal when it comes to spending money on movies; he buys them the day they're released, regardless of price.",Nero,\N,2010-12-09 21:55:17
276921,eng,No inequality should be allowed to exist between men and women.,CM,\N,2012-03-28 03:53:32
18429,eng,What are the banking hours?,Zifre,\N,2011-04-18 00:52:08
19393,eng,A huge federal budget deficit has been plaguing the American economy for many years.,CK,\N,2010-10-18 00:17:18
1065,deu,Wir brauchen einen Krankenwagen.,MUIRIEL,\N,2010-10-14 20:51:46
58815,eng,This increase in unemployment is a consequence of the recession.,CK,\N,2013-12-07 10:36:25
54196,eng,Service economy is a useful labor that does not produce a tangible commodity.,\N,\N,\N


In [1408]:
%%time
import numpy as np
my_list = []
anki = pd.DataFrame({"english": [],"spanish": [],"example":[],"translation":[]})
for word in economics_vocab_no_cognates.index[:50]:
#for word in ["macroeconomic"]:
    try:
        definition = np.nan
        example = np.nan
        example_trans = np.nan
        
        url = "https://www.wordreference.com/es/translation.asp?tranword={}".format(word)
        content = requests.get(url).content
        soup = BeautifulSoup(content,'lxml')
        definition = soup.select(".ToWrd")[1].text.split()[0]
        example_tag = soup.select(".FrEx")[0]
        example = example_tag.text
        if example_tag.parent.nextSibling.nextSibling.select(".ToEx")[0]:
            example_trans = example_tag.parent.nextSibling.nextSibling.select(".ToEx")[0].text
        else:
            example_trans = "no translation found"
        tmp_df = pd.DataFrame({"english": [word],"spanish": [definition],"example":[example],"translation":[example_trans]})
        anki = anki.append(tmp_df)
    except:
        try:
            tmp_df = pd.DataFrame({"english": [word],"spanish": [definition],"example":[example],"translation":[np.nan]})
            anki = anki.append(tmp_df)
        except:
            try:
                tmp_df = pd.DataFrame({"english": [word],"spanish": [definition],"example":[np.nan],"translation":[np.nan]})
                anki = anki.append(tmp_df)
            except:
                try:
                    tmp_df = pd.DataFrame({"english": [word],"spanish": [np.nan],"example":[np.nan],"translation":[np.nan]})
                    anki = anki.append(tmp_df)
                except:
                    pass
        
        

CPU times: user 3.58 s, sys: 234 ms, total: 3.81 s
Wall time: 18.3 s


In [1362]:
example_tag.parent.nextSibling.nextSibling.select(".ToEx")[0]

<td class="ToEx" colspan="2">La globalización significa que debemos competir con trabajadores de todo el mundo.</td>

In [1466]:
anki["cloze"] = 0
anki = anki.dropna(axis=0)
for i in range(len(anki)):
    regex = r"{}\S*".format(anki.iloc[i,0][:-2])
    match = re.findall(regex,anki.iloc[i,2].lower())[0]
    regex2 = r" {}\S*".format(anki.iloc[i,1][:-4].lower())
    if re.search(regex2,anki.iloc[i,3].lower()):
        match2 = re.findall(regex2,anki.iloc[i,3].lower())[0]
    else:
        match2 = anki.iloc[i,1]
    anki.iloc[i,4] = re.sub(match,"{{{{c1::{}::{}}}}}".format(match,match2),anki.iloc[i,2].lower())


In [1467]:
anki

,english,spanish,example,translation,cloze
0,currency,moneda,I need to get some foreign currency for my holidays.,Tengo que conseguir moneda extranjera para mis vacaciones.,i need to get some foreign {{c1::currency:: moneda}} for my holidays.
0,poverty,pobreza,He has a tiny income and lives in great poverty.,Tiene ingresos muy reducidos y vive en la pobreza.,he has a tiny income and lives in great {{c1::poverty.:: pobreza.}}
0,spending,gastos,Robert buys more than he can afford every month; his spending is out of control.,"Todos los meses, Robert compra más cosas de las que se puede permitir. Sus gastos están fuera de control.",robert buys more than he can afford every month; his {{c1::spending:: gastos}} is out of control.
0,inequality,injusticia,The party's main aim is to eliminate inequality in society.,El objetivo principal del partido es eliminar la injusticia en la sociedad.,the party's main aim is to eliminate {{c1::inequality:: injusticia}} in society.
0,banking,banca,Banking as an industry is struggling right now.,La banca es una industria que en estos momentos tiene dificultades.,{{c1::banking:: banca}} as an industry is struggling right now.
0,deficit,déficit,The deficit in next year's budget is expected to be lower.,Se espera que el déficit en el presupuesto del año que viene sea inferior.,the {{c1::deficit:: déficit}} in next year's budget is expected to be lower.
0,unemployment,desempleo,"Unemployment fell for a third month, encouraging economists.","El desempleo cayó por tercer mes consecutivo, alentando a los economistas.","{{c1::unemployment:: desempleo}} fell for a third month, encouraging economists."
0,commodity,producto,The country is famous for commodities such as textiles and grains.,"El país es famoso por sus productos básicos, tales como textiles y granos.",the country is famous for {{c1::commodities:: productos}} such as textiles and grains.
0,framework,armazón,"After completing the building, they remove the supporting framework.","Cuando terminaron el edificio, quitaron el armazón de soporte.","after completing the building, they remove the supporting {{c1::framework.:: armazón}}"
0,financing,financiamiento,We need to carefully plan some fund-raisers for the financing of the project. An anonymous donor has volunteered to be responsible for the financing of the project.,Necesitamos planificar con detalle cómo recaudar fondos para el financiamiento del proyecto. Un donante anónimo se ha ofrecido como voluntario para el financiamiento del proyecto.,we need to carefully plan some fund-raisers for the {{c1::financing:: financiamiento}} of the project. an anonymous donor has volunteered to be responsible for the {{c1::financing:: financiamiento}} of the project.


In [1468]:
anki[["cloze","translation"]].to_csv("to_anki.csv",sep="\t",header=None,index=False)

In [1443]:
regex = r"{}\S*".format(anki.iloc[i,0][:-2])
re.findall(match,anki.iloc[i,2].lower())[0]

'banking'

In [1439]:
anki.iloc[i,2]


'Banking as an industry is struggling right now.'

In [1359]:
economics_vocab_no_cognates.index[:20]

Index(['currency', 'poverty', 'spending', 'inequality', 'banking',
       'macroeconomic', 'deficit', 'wage', 'unemployment', 'commodity',
       'output', 'framework', 'financing', 'euro', 'policymakers',
       'strengthen', 'governance', 'regulatory', 'employment',
       'globalization'],
      dtype='object')

In [1360]:
anki

,english,spanish,example,translation
0,currency,moneda,I need to get some foreign currency for my holidays.,Tengo que conseguir moneda extranjera para mis vacaciones.
0,poverty,pobreza,He has a tiny income and lives in great poverty.,Tiene ingresos muy reducidos y vive en la pobreza.
0,spending,gastos,Robert buys more than he can afford every month; his spending is out of control.,"Todos los meses, Robert compra más cosas de las que se puede permitir. Sus gastos están fuera de control."
0,inequality,injusticia,The party's main aim is to eliminate inequality in society.,El objetivo principal del partido es eliminar la injusticia en la sociedad.
0,banking,banca,Banking as an industry is struggling right now.,La banca es una industria que en estos momentos tiene dificultades.
0,deficit,déficit,The deficit in next year's budget is expected to be lower.,Se espera que el déficit en el presupuesto del año que viene sea inferior.
0,unemployment,desempleo,"Unemployment fell for a third month, encouraging economists.","El desempleo cayó por tercer mes consecutivo, alentando a los economistas."
0,commodity,producto,The country is famous for commodities such as textiles and grains.,"El país es famoso por sus productos básicos, tales como textiles y granos."
0,framework,armazón,"After completing the building, they remove the supporting framework.","Cuando terminaron el edificio, quitaron el armazón de soporte."
0,financing,financiamiento,We need to carefully plan some fund-raisers for the financing of the project. An anonymous donor has volunteered to be responsible for the financing of the project.,Necesitamos planificar con detalle cómo recaudar fondos para el financiamiento del proyecto. Un donante anónimo se ha ofrecido como voluntario para el financiamiento del proyecto.


'gastos'

In [1270]:
my_list = []
for word in economics_vocab_no_cognates.index[:10]:
    try:
        url = "https://www.linguee.com/english-spanish/search?source=auto&query={}".format(word)
        content = requests.get(url).content
        soup = BeautifulSoup(content,'lxml')
        example1 = soup.select(".tag_s")[0].text
        example2 = soup.select(".tag_t")[0].text
        translation = soup.select(".dictLink.featured")[0].text
        my_list.append((word,example1,example2,translation))
    except:
        continue

In [1564]:
url="https://www.imf.org/en/news/articles/2015/09/28/04/53/sonew101015a"
content = requests.get(url).content
soup = BeautifulSoup(content,'lxml')
english_t = soup.body.text

In [1727]:
# Get the pdfs from the imf website

cmd = "mkdir pdfs"
os.system(cmd)
for year in range(2009,2018):
    for month in ["03","06","09","12"]:
        short_year = str(year)[-2:]
        cmd = """curl "https://www.imf.org/external/pubs/ft/fandd/{}/{}/pdf/fd{}{}.pdf" \
                --output ./pdfs/eng-{}-{}.pdf""".format(year,month,month,short_year,year,month)
        os.system(cmd)
        cmd = """curl "https://www.imf.org/external/pubs/ft/fandd/spa/{}/{}/pdf/fd{}{}s.pdf" \
                --output ./pdfs/spa-{}-{}.pdf""".format(year,month,month,short_year,year,month)
        os.system(cmd)

Using hunalign
D. Varga, L. Németh, P. Halácsy, A. Kornai, V. Trón, V. Nagy (2005).
Parallel corpora for medium density languages
In Proceedings of the RANLP 2005

In [7]:
# Create the sentences aligned parallel corpus

%%time
import pandas as pd
import nltk
import os
sentence_df = pd.DataFrame({"english":[],"spanish":[]})
cmd = "rm -rf tmp ; mkdir tmp"
os.system(cmd)
for year in range(2009,2018):
    for month in ["03","06","09","12"]:
        cmd = """xpdf-tools-mac-4.00/bin64/pdftotext \
        ./pdfs/eng-{}-{}.pdf  tmp/eng-{}-{}.txt""".format(year,month,year,month)
        os.system(cmd)
        cmd = """xpdf-tools-mac-4.00/bin64/pdftotext \
        ./pdfs/spa-{}-{}.pdf  tmp/spa-{}-{}.txt""".format(year,month,year,month)
        os.system(cmd)
        e = open("tmp/eng-{}-{}.txt".format(year,month),"r",encoding="Latin-1").read()
        s = open("tmp/spa-{}-{}.txt".format(year,month),"r",encoding="Latin-1").read()
        english_sentences = nltk.sent_tokenize(e)
        spanish_sentences = nltk.sent_tokenize(s)
        eo = open("tmp/sent-eng-{}-{}.txt".format(year,month),"w")
        for line in english_sentences:
            eo.write((line+"\n"))
        eo.close()
        so = open("tmp/sent-spa-{}-{}.txt".format(year,month),"w")
        for line in spanish_sentences:
            so.write((line+"\n"))
        so.close()
        
        #Aligning sentences
        cmd = """hunalign/hunalign-1.2/src/hunalign/hunalign \
        hunalign/hunalign-1.2/data/null.dic \
        tmp/sent-eng-{}-{}.txt tmp/sent-spa-{}-{}.txt\
        -text -bisent > tmp/aligned-{}-{}.txt""".format(year,month,year,month,year,month)
        os.system(cmd)

        #putting lines into dataframe
        tmp_df = pd.DataFrame({"english":[],"spanish":[]})
        lines = open("tmp/aligned-{}-{}.txt".format(year,month),"r").readlines()
        for line in lines:
            sentences = line.split("\t")
            eng = sentences[0]
            spa = sentences[1]
            tmp_df = tmp_df.append(pd.DataFrame({"english":[eng],"spanish":[spa]}))
        sentence_df = sentence_df.append(tmp_df)
        
sentence_df.to_csv("imf_sentences.txt",sep="\t",index=False,header=None)

CPU times: user 2min 28s, sys: 2.15 s, total: 2min 30s
Wall time: 5min 58s


In [20]:
economics_vocab_list = open("economics_vocab.txt","r").readlines()
economics_vocab_list = pd.Series(economics_vocab)
economics_vocab_list = economics_vocab.iloc[4:]
economics_vocab_list = economics_vocab.str.rstrip()

In [26]:
%%time
import requests
from bs4 import BeautifulSoup
import numpy as np
definitions = pd.DataFrame({"word":[],"definitions":[]})
for word in economics_vocab_list:
    try:
        url="http://www.spanishdict.com/translate/{}".format(word)
        content = requests.get(url).content
        soup = BeautifulSoup(content,'lxml')
        entry = str(soup.select(".dictionary-entry")[0])
        tmp_df = pd.DataFrame({"word":[word],"definitions":[entry]})
        definitions = definitions.append(tmp_df)
    except:
        tmp_df = pd.DataFrame({"word":[word],"definitions":[np.nan]})
        definitions = definitions.append(tmp_df)
definitions.to_csv("defintions_4_anki.txt",header=None,index=False,sep="\t")

CPU times: user 1min 33s, sys: 3.17 s, total: 1min 36s
Wall time: 16min 8s


In [31]:
to_anki = definitions.copy()

In [33]:
to_anki.index = to_anki['word']

In [63]:
%%time
for word in to_anki.index[:100]:
    try:
        sentence_df['sentence_length'] = sentence_df['english'].apply(lambda x: len(x.split()))
        examples = (sentence_df[sentence_df['english']
             .str.contains(word)].sort_values('sentence_length'))
        examples = examples[['english','spanish']].loc[examples['sentence_length'] > 15]
        examples = examples.loc[~examples['english'].str.contains("\d") & ~examples['spanish'].str.contains("\d")]
        examples = examples.iloc[:5,:]
        for i in range(5):
            to_anki.loc[word,"ex{}_eng".format(i)] = examples.iloc[i,0]
            to_anki.loc[word,"ex{}_spa".format(i)] = examples.iloc[i,1]
    except:
        for i in range(5):
            to_anki.loc[word,"ex{}_eng".format(i)] = np.nan
            to_anki.loc[word,"ex{}_spa".format(i)] = np.nan

CPU times: user 28.4 s, sys: 210 ms, total: 28.7 s
Wall time: 28.9 s


In [65]:
to_anki.iloc[:100,4]

word
currency           ·Help ameliorate the problems in manufacturing...
poverty            How can we translate these guiding principles ...
spending           Canada and France rely on such controls and ar...
inequality         Moreover, the economic and social costs associ...
banking            Cost measures of economies' banking systems ar...
macroeconomic      To see how these broad patterns of macroeconom...
deficit            The prevalent feeling was that fiscal rules--e...
wage               Second, the structure of product and labor mar...
unemployment       to export unemployment by blocking imports Ins...
commodity          In the past year emerging markets have suffere...
output             Productivity growth is crucial to increasing e...
framework          And if you pursue a more prudent economic poli...
financing          Private participation in infrastructure has su...
euro               After a fast start, market use of the euro has...
policymakers       If policym

In [66]:
import re
for word in to_anki.index[:100]:
    try:
        regex = r"{}\S*".format(to_anki.loc[word,'word'][:-2])
        match = re.findall(regex,to_anki.loc[word,'ex0_eng'].lower())[0]
        to_anki.loc[word,'ex0_cloze'] = re.sub(match,"{{{{c1::{}}}}}".format(match),to_anki.loc[word,'ex0_eng'].lower())
    except:
        continue

In [67]:
to_anki

,word,definitions,ex0_eng,ex0_spa,ex1_eng,ex1_spa,ex2_eng,ex2_spa,ex3_eng,ex3_spa,ex4_eng,ex4_spa,ex0_cloze
word,,,,,,,,,,,,,
currency,currency,"<div class=""dictionary-entry dictionary-neodic...",But what goes up does not come down so easily ...,Pero lo que sube no baja con tanta facilidad s...,·Help ameliorate the problems in manufacturing...,· Ayudar a aliviar los problemas que causa la ...,"On the other hand, when a currency's value ris...","Por otra parte, cuando aumenta el valor de la ...",The perception that lending anywhere within th...,La percepción de que los créditos dentro de la...,Financial market development in the home count...,El grado de desarrollo del mercado financiero ...,but what goes up does not come down so easily ...
poverty,poverty,"<div class=""dictionary-entry dictionary-neodic...",She wants it to be part of a data revolution t...,Quiere que sea parte de una revolución en mate...,How can we translate these guiding principles ...,¿Cómo podemos traducir estos principios rector...,The catalytic role of infrastructure in povert...,El papel catalizador de la infraestructura en ...,"In addition to extreme poverty, low-income cou...","Además de la pobreza extrema, los países de ba...","People living at or under the poverty line, wi...",Quienes viven en la línea de pobreza o por deb...,she wants it to be part of a data revolution t...
spending,spending,"<div class=""dictionary-entry dictionary-neodic...","Canada, meanwhile, implemented profound struct...","Canadá, entre tanto, puso en marcha profundas ...",Canada and France rely on such controls and ar...,Canadá y Francia aplican esos controles y,Public health spending as a percent of GDP has...,"El gasto en salud pública ha aumentado, como p...","If commodity prices fall at a later date, spen...","Si después los precios caen, es posible que se...","By increasing spending on infrastructure, Germ...","Al aumentar el gasto en infraestructura, Alema...","canada, meanwhile, implemented profound struct..."
inequality,inequality,"<div class=""dictionary-entry dictionary-neodic...",There are many channels through which opening ...,de capital puede incrementar la,"Moreover, the economic and social costs associ...","Además, los costos económicos y sociales del d...",These results are consistent with macroeconomi...,Estos resultados concuerdan con los datos macr...,"Often, researchers want to compare inequality ...",A menudo los investigadores quieren comparar l...,It also looks at inclusive growth policies to ...,"Asimismo, evalúa políticas de crecimiento incl...",there are many channels through which opening ...
banking,banking,"<div class=""dictionary-entry dictionary-neodic...","In fact, Caribbean countries have been among t...","De hecho, los países del Caribe han sido de lo...",Cost measures of economies' banking systems ar...,Los indicadores de los costos de los sistemas ...,The primary cause of the current financial cri...,La principal causa de la actual crisis financi...,They are profit-maximizing intermediaries betw...,Nuestro estudio identifica las fuentes de la e...,Strong supervision at the euro area level must...,La supervisión en la zona del euro debe contro...,"in fact, caribbean countries have been among t..."
macroeconomic,macroeconomic,"<div class=""dictionary-entry dictionary-collin...","To be effective, regulations should provide in...","Para ser eficaces, las normas deben alentar a ...",To see how these broad patterns of macroeconom...,"En promedio, la generación de adultos jóvenes ...",MIT was then at the forefront of the developme...,MIT iba entonces a la vanguardia de la creació...,"However, to be successful, any such reform mus...","No obstante, para ser eficaz, esta reforma deb...",Many countries have slashed their public debt ...,Muchos países recortaron drásticamente sus coe...,"to be effective, regulations should provide in..."
deficit,deficit,"<div class=""dictionary-entry dictionary-neodic...","People often think that the program of gro

In [72]:
to_anki.iloc[:100,[12,3,1]].to_csv("examples_anki.txt",index=False,header=None,sep="\t")
    

In [71]:
pd.options.display.max_colwidth=1000
tmp = to_anki.iloc[:100,[12,3,1]]
tmp.head(20)

,ex0_cloze,ex0_spa,definitions
word,,,
currency,but what goes up does not come down so easily when there is no independent {{c1::currency.}},Pero lo que sube no baja con tanta facilidad si la moneda no es independiente.,"<div class=""dictionary-entry dictionary-neodict""><div class=""dictionary-neodict-entry-title"">currency</div><a class=""has-tooltip dictionary-neodict-first-part-of-speech part_of_speech"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""A noun is a word referring to a person, animal, place, thing, feeling or idea (e.g. man, dog, house)."">noun</a><div class=""dictionary-neodict-indent-1""><span class=""def"">1. </span><span class=""context"">(finance)</span> <div class=""dictionary-neodict-indent-2""><div class=""dictionary-neodict-translation""><span class=""dictionary-neodict-translation-letters"">a. </span><a class=""dictionary-neodict-translation-translation"" href=""/translate/la%20moneda"">la moneda</a><span class=""def""> </span><a class=""has-tooltip def"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""(f) means that a noun is feminine. Spanish nouns have a gender, which is either feminine (like la ..."
poverty,she wants it to be part of a data revolution to guide the fight against {{c1::poverty.}},Quiere que sea parte de una revolución en materia de datos que oriente la lucha contra la pobreza.,"<div class=""dictionary-entry dictionary-neodict""><div class=""dictionary-neodict-entry-title"">poverty</div><a class=""has-tooltip dictionary-neodict-first-part-of-speech part_of_speech"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""A noun is a word referring to a person, animal, place, thing, feeling or idea (e.g. man, dog, house)."">noun</a><div class=""dictionary-neodict-indent-1""><span class=""def"">1. </span><span class=""context"">(state of being poor)</span> <div class=""dictionary-neodict-indent-2""><div class=""dictionary-neodict-translation""><span class=""dictionary-neodict-translation-letters"">a. </span><a class=""dictionary-neodict-translation-translation"" href=""/translate/la%20pobreza"">la pobreza</a><span class=""def""> </span><a class=""has-tooltip def"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""(f) means that a noun is feminine. Spanish nouns have a gender, which is either femin..."
spending,"canada, meanwhile, implemented profound structural reforms in {{c1::spending}} and tax policy that had a longer-lasting impact.","Canadá, entre tanto, puso en marcha profundas reformas estructurales de la política de gasto y tributación que tuvieron un impacto más duradero.","<div class=""dictionary-entry dictionary-neodict""><div class=""dictionary-neodict-entry-title"">spending</div><a class=""has-tooltip dictionary-neodict-first-part-of-speech part_of_speech"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""A noun is a word referring to a person, animal, place, thing, feeling or idea (e.g. man, dog, house)."">noun</a><div class=""dictionary-neodict-indent-1""><span class=""def"">1. </span><span class=""context"">(finance)</span> <div class=""dictionary-neodict-indent-2""><div class=""dictionary-neodict-translation""><span class=""dictionary-neodict-translation-letters"">a. </span><a class=""dictionary-neodict-translation-translation"" href=""/translate/los%20gastos"">los gastos</a><span class=""def""> </span><a class=""has-tooltip def"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""(m) means that a noun is masculine. Spanish nouns have a gender, which is either feminine (like ..."
inequality,there are many channels through which opening up the capital account can lead to higher {{c1::inequality.}},de capital puede incrementar la,"<div class=""dictionary-entry

In [1792]:
concat = []
a = (str(soup.select(".dictionary-neodict-indent-1")[0]) + str(soup.select(".dictionary-neodict-indent-1")[1]))

In [1827]:
tmp = pd.DataFrame(definitions['definitions'])
tmp.index = definitions['word']
tmp.head()

,definitions
word,
currency,"<div class=""dictionary-entry dictionary-neodict""><div class=""dictionary-neodict-entry-title"">currency</div><a class=""has-tooltip dictionary-neodict-first-part-of-speech part_of_speech"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""A noun is a word referring to a person, animal, place, thing, feeling or idea (e.g. man, dog, house)."">noun</a><div class=""dictionary-neodict-indent-1""><span class=""def"">1. </span><span class=""context"">(finance)</span> <div class=""dictionary-neodict-indent-2""><div class=""dictionary-neodict-translation""><span class=""dictionary-neodict-translation-letters"">a. </span><a class=""dictionary-neodict-translation-translation"" href=""/translate/la%20moneda"">la moneda</a><span class=""def""> </span><a class=""has-tooltip def"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""(f) means that a noun is feminine. Spanish nouns have a gender, which is either feminine (like la mujer or la luna) or masculine (like el hombre or el sol)."">(F)</a> </div><div class=""dictionary-neodict-indent-3""><div class=""dictionary-neodict-example""><span>The currency of the European Union is the euro.</span><span class=""icon-dash""></span><em class=""exB"">La moneda de la Unión Europea es el euro.</em></div></div><div class=""dictionary-neodict-translation""><span class=""dictionary-neodict-translation-letters"">b. </span><a class=""dictionary-neodict-translation-translation"" href=""/translate/la%20divisa"">la divisa</a><span class=""def""> </span><a class=""has-tooltip def"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""(f) means that a noun is feminine. Spanish nouns have a gender, which is either feminine (like la mujer or la luna) or masculine (like el hombre or el sol)."">(F)</a> </div><div class=""dictionary-neodict-indent-3""><div class=""dictionary-neodict-example""><span>How does the dollar's performance compare to other currencies?</sp..."
poverty,"<div class=""dictionary-entry dictionary-neodict""><div class=""dictionary-neodict-entry-title"">poverty</div><a class=""has-tooltip dictionary-neodict-first-part-of-speech part_of_speech"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""A noun is a word referring to a person, animal, place, thing, feeling or idea (e.g. man, dog, house)."">noun</a><div class=""dictionary-neodict-indent-1""><span class=""def"">1. </span><span class=""context"">(state of being poor)</span> <div class=""dictionary-neodict-indent-2""><div class=""dictionary-neodict-translation""><span class=""dictionary-neodict-translation-letters"">a. </span><a class=""dictionary-neodict-translation-translation"" href=""/translate/la%20pobreza"">la pobreza</a><span class=""def""> </span><a class=""has-tooltip def"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""(f) means that a noun is feminine. Spanish nouns have a gender, which is either feminine (like la mujer or la luna) or masculine (like el hombre or el sol)."">(F)</a> </div><div class=""dictionary-neodict-indent-3""><div class=""dictionary-neodict-example""><span>The number of children living in poverty decreased last year.</span><span class=""icon-dash""></span><em class=""exB"">El número de niños que viven en la pobreza disminuyó el año pasado.</em></div></div></div></div><div class=""dictionary-neodict-indent-1""><span class=""def"">2. </span><span class=""context"">(scarcity)</span> <div class=""dictionary-neodict-indent-2""><div class=""dictionary-neodict-translation""><span class=""dictionary-neodict-translation-letters"">a. </span><a class=""dictionary-neodict-translation-translation"" href=""/translate/la%20pobreza"">la pobreza</a><span class=""def""> </span><a class=""has-tooltip def"" data-toggle=""tooltip"" href=""http:/

In [1829]:
w_defs = economics_vocab_no_cognates.copy()
w_defs = w_defs.merge(tmp,left_index=True,right_index=True,how="left")

In [1873]:
w_defs.iloc[:10]

,imf_rank,imf_freq,general_rank,definitions,ex1,ex1_eng,ex1_spa,ex0_eng,ex0_spa,ex2_eng,ex2_spa,ex3_eng,ex3_spa,ex4_eng,ex4_spa
currency,136,2328,3297,"<div class=""dictionary-entry dictionary-neodict""><div class=""dictionary-neodict-entry-title"">currency</div><a class=""has-tooltip dictionary-neodict-first-part-of-speech part_of_speech"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""A noun is a word referring to a person, animal, place, thing, feeling or idea (e.g. man, dog, house)."">noun</a><div class=""dictionary-neodict-indent-1""><span class=""def"">1. </span><span class=""context"">(finance)</span> <div class=""dictionary-neodict-indent-2""><div class=""dictionary-neodict-translation""><span class=""dictionary-neodict-translation-letters"">a. </span><a class=""dictionary-neodict-translation-translation"" href=""/translate/la%20moneda"">la moneda</a><span class=""def""> </span><a class=""has-tooltip def"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""(f) means that a noun is feminine. Spanish nouns have a gender, which is either feminine (like la mujer or la luna) or masculine (like el hombre or el sol)."">(F)</a> </div><div class=""dictionary-neodict-indent-3""><div class=""dictionary-neodict-example""><span>The currency of the European Union is the euro.</span><span class=""icon-dash""></span><em class=""exB"">La moneda de la Unión Europea es el euro.</em></div></div><div class=""dictionary-neodict-translation""><span class=""dictionary-neodict-translation-letters"">b. </span><a class=""dictionary-neodict-translation-translation"" href=""/translate/la%20divisa"">la divisa</a><span class=""def""> </span><a class=""has-tooltip def"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""(f) means that a noun is feminine. Spanish nouns have a gender, which is either feminine (like la mujer or la luna) or masculine (like el hombre or el sol)."">(F)</a> </div><div class=""dictionary-neodict-indent-3""><div class=""dictionary-neodict-example""><span>How does the dollar's performance compare to other currencies?</sp...",NaN,Changing a national currency is a highly political decision.,Cambiar de moneda es una decisión sumamente política.,Q: How can I buy bitcoin or other cryptocurrency?,P: ¿Cómo puedo comprar bitcoines u otra criptomoneda?,"massive counterfeiting of the existing currency, or even war.",falsificación masiva de la moneda en circulación o incluso la guerra.,(percent of foreign currency deposits to total bank deposits),(porcentaje de depósitos en moneda extranjera respecto de los depósitos bancarios totales),This two-currency simulation should not be taken too literally.,"El cambio de moneda de reserva es lento, pero llega un momento en que es inevitable."
poverty,146,2123,2080,"<div class=""dictionary-entry dictionary-neodict""><div class=""dictionary-neodict-entry-title"">poverty</div><a class=""has-tooltip dictionary-neodict-first-part-of-speech part_of_speech"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""A noun is a word referring to a person, animal, place, thing, feeling or idea (e.g. man, dog, house)."">noun</a><div class=""dictionary-neodict-indent-1""><span class=""def"">1. </span><span class=""context"">(state of being poor)</span> <div class=""dictionary-neodict-indent-2""><div class=""dictionary-neodict-translation""><span class=""dictionary-neodict-translation-letters"">a. </span><a class=""dictionary-neodict-translation-translation"" href=""/translate/la%20pobreza"">la pobreza</a><span class=""def""> </span><a class=""has-tooltip def"" data-toggle=""tooltip"" href=""http://www.spanishdict.com/guide/masculine-and-feminine-nouns"" title=""(f) means that a noun is feminine. Spanish nouns have a gender, which is either feminine (like la mujer or la luna) or masculine (like el hombre or

### Cells not currently in use

In [1617]:
# Links for articles on the imf news website
# Not currently using these but could be a nice additional resource

found_links = []

for i in range(97): 
    url="https://www.imf.org/es/News/Search?datefrom=1994-01-01&dateto=2018-11-02&page={}".format(i)
    content = requests.get(url).content
    soup = BeautifulSoup(content,'lxml')
    for link in soup.find_all("a"):
        if link.get("href"):
            if re.search("es/news/articles",link['href'].lower()):
                if link['href'] not in found_links:
                    found_links.append(link['href'])

    

In [1544]:
# Tried to get definitions from Linguee. Linguee blocked me for sending too many requests

for i, word in enumerate(economics_vocab_no_cognates.index[:50]):
    try:
        url = "https://www.linguee.com/english-spanish/search?source=auto&query={}".format(word)
        content = requests.get(url).content
        soup = BeautifulSoup(content,'lxml')
        #soup.find_all(True, {'class':['sentence', 'left']},text="imf.org")
        print(i,word)
        print(soup.find_all(text="imf.org")[0].parent.parent.parent.parent.select(".sentence.left")[0].text)
        print(soup.find_all(text="imf.org")[0].parent.parent.parent.parent.select(".sentence.right2")[0].text)
    except:
        continue

0 currency
1 poverty
2 spending
3 inequality
4 banking
5 macroeconomic
6 deficit
7 wage
8 unemployment
9 commodity
10 output
11 framework
12 financing
13 euro
14 policymakers
15 strengthen
16 governance
17 regulatory
18 employment
19 globalization
20 liberalization
21 enterprise
22 moreover
23 inflow
24 liquidity
25 expenditure
26 lending
27 equity
28 further
29 donor
30 boost
31 indicator
32 tariff
33 enhance
34 trading
35 instance
36 wealth
37 volatility
38 boom
39 creditor
40 remittance
41 imbalance
42 transparency
43 constraint
44 arrangement
45 surplus
46 sustainable
47 scheme
48 borrow
49 liability
